In [ ]:
import sys, os
sys.path.extend([os.path.abspath('../')])

In [ ]:
import numpy as np
import sympy as sy
import sparse as sp
import math

In [ ]:
from qgs.params.params import QgParams
from qgs.functions.tendencies import create_tendencies

In [ ]:
model_parameters = QgParams(dynamic_T=True)

In [ ]:
model_parameters.set_atmospheric_channel_fourier_modes(2, 2, mode="symbolic")
# Mode truncation at the wavenumber 2 in the x and at the 
# wavenumber 4 in the y spatial coordinates for the ocean
model_parameters.set_oceanic_basin_fourier_modes(2, 4, mode="symbolic")

In [ ]:
model_parameters.symbolic_params

In [ ]:
from qgs.inner_products.symbolic import AtmosphericSymbolicInnerProducts, OceanicSymbolicInnerProducts
from qgs.tensors.qgtensor import QgsTensor, QgsTensorDynamicT, QgsTensorT4
from qgs.tensors.symbolic_qgtensor import SymbolicTensorLinear, SymbolicTensorDynamicT
from qgs.tensors.symbolic_qgtensor import _symbolic_tensordot, _shift_dict_keys, _add_to_dict

In [ ]:
%%time
# atm_ip = AtmosphericSymbolicInnerProducts(model_parameters)
# ocn_ip_stored = OceanicSymbolicInnerProducts(model_parameters, stored=True, return_symbolic=True)
ocn_ip = OceanicSymbolicInnerProducts(model_parameters, stored=False, return_symbolic=True)

In [ ]:
%%time
# atm_ip_stored = AtmosphericSymbolicInnerProducts(model_parameters, return_symbolic=True)
atm_ip = AtmosphericSymbolicInnerProducts(model_parameters, stored=False, return_symbolic=True)

In [ ]:
# atm_ip_stored.save_to_file('temp_atm_sym_de.ip')
# ocn_ip_stored.save_to_file('temp_ocn_sym_de.ip')

In [ ]:
atm_loaded = AtmosphericSymbolicInnerProducts(model_parameters, stored=False, return_symbolic=True)
ocn_loaded = OceanicSymbolicInnerProducts(model_parameters, stored=False, return_symbolic=True)

In [ ]:
atm_loaded.load_from_file('temp_atm_sym_de.ip')
ocn_loaded.load_from_file('temp_ocn_sym_de.ip')

## Testing the symbolic_qgtensor

In [ ]:
ten = SymbolicTensorDynamicT(params=model_parameters, atmospheric_inner_products=atm_loaded, oceanic_inner_products=ocn_loaded)

In [ ]:
%%time
ten.compute_tensor()

In [ ]:
ten.tensor.shape

In [ ]:
_, _, num_ten = create_tendencies(model_parameters, return_qgtensor=True)

In [ ]:
subs_list = ten.test_tensor_numerically(tol=1e-12)

In [ ]:
ten.print_tensor(np.array(ten.subs_tensor()).astype(np.float64))

In [ ]:
ten.print_tensor(num_ten.tensor.todense())